In [102]:
from openai import OpenAI
import numpy as np
import json
import faiss

In [103]:
#load json files
def load_documents(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    return data

# Generate embeddings for each document
def get_embedding(text):
    response = client.embeddings.create(model = "text-embedding-ada-002", input = text)
    embedding_array = np.array(response.data[0].embedding)
    
    return embedding_array


In [104]:
with open("C:/Users/rshua/keys/secret_key.json", "r") as file:
    secrets = json.load(file)
    api_key = secrets["OPENAI_API_KEY"]

client = OpenAI(api_key = api_key)

In [105]:
data = load_documents("format_correct_TEST2.json")

In [106]:
# data.items()

In [107]:
# Convert JSON to a list of text documents
documents = []

for week_data in data["Weeks"]:
    week = week_data["week"]
    for position, position_data in week_data["Positions"].items():
        for player_name, player_stats in position_data["Players"].items():
            text = f"Week: {week}, Position: {position}, Player: {player_name}, Team: {player_stats['team']}, "
            text += f"Dropbacks: {player_stats['dropbacks']}, Attempts: {player_stats['att']}, Completions: {player_stats['comp']}, "
            text += f"Yards: {player_stats['yds']}, TDs: {player_stats['tds']}, INTs: {player_stats['ints']}, "
            text += f"Sacks: {player_stats['sks']}, Rush Carries: {player_stats['rushCarries']}, Rush Yards: {player_stats['rushYds']}, "
            text += f"Rush TDs: {player_stats['rushTds']}, Fumbles: {player_stats['fumbles']}, Fantasy Points: {player_stats['fantasyPts']}"
            documents.append({"text": text})
            print(week)

last_week = week
next_week = week + 1


1
1
2
2
3
3


In [108]:
for doc in documents:
    print(doc["text"])

Week: 1, Position: Quarterback, Player: Josh Allen, Team: BUF, Dropbacks: 30, Attempts: 23, Completions: 18, Yards: 232, TDs: 2, INTs: 0, Sacks: 2, Rush Carries: 9, Rush Yards: 39, Rush TDs: 2, Fumbles: 1, Fantasy Points: 33.2
Week: 1, Position: Quarterback, Player: Baker Mayfield, Team: TB, Dropbacks: 34, Attempts: 30, Completions: 24, Yards: 289, TDs: 4, INTs: 0, Sacks: 1, Rush Carries: 3, Rush Yards: 21, Rush TDs: 0, Fumbles: 0, Fantasy Points: 29.7
Week: 2, Position: Quarterback, Player: Josh Allen, Team: BUF, Dropbacks: 20, Attempts: 19, Completions: 13, Yards: 139, TDs: 1, INTs: 0, Sacks: 0, Rush Carries: 2, Rush Yards: 2, Rush TDs: 0, Fumbles: 1, Fantasy Points: 9.8
Week: 2, Position: Quarterback, Player: Baker Mayfield, Team: TB, Dropbacks: 28, Attempts: 19, Completions: 12, Yards: 185, TDs: 1, INTs: 1, Sacks: 5, Rush Carries: 5, Rush Yards: 34, Rush TDs: 1, Fumbles: 2, Fantasy Points: 19.8
Week: 3, Position: Quarterback, Player: Josh Allen, Team: BUF, Dropbacks: 34, Attempts: 

In [109]:
# Create a list of text documents and their embeddings
# texts = [doc["text"] for doc in documents]
texts = []

for i in documents:
    text_content = i["text"]
    texts.append(text_content)

embeddings = np.array([get_embedding(text) for text in texts])

# Store embeddings in FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(np.array([embeddings]))
# faiss.write_index(index, "vector_store.index")



In [110]:
print(embeddings.shape)

(6, 1536)


In [111]:
print(dimension)

1536


In [112]:
# Function to retrieve the most relevant documents
def retrieve_relevant_docs(query, top_k=2):
    query_embedding = get_embedding(query)
    query_embedding = query_embedding.reshape(1, -1)

    distances, indices = index.search(query_embedding, top_k)

    relevant_docs = []

    for i in indices[0]:
        relevant_docs.append(texts[i])

    return relevant_docs


    # query_embedding = get_embedding(query)
    # distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    
    # return [texts[i] for i in indices[0]]



In [113]:
user_query = "How did Baker Mayfield do in Week 1? "

retrieved_docs = retrieve_relevant_docs(user_query)
context = "\n".join(retrieved_docs)


In [114]:
system_message = (
    "You are a helpful Fantasy Football assistant specializing in providing fantasy football advice. You assist with:"
    "Summarizing NFL player statistics based on retrieved information"
    "Providing valuable line up suggestions like start/sit recommendations"
    "Providing insightful trade advice"

    "If asked about information from a certain week, always mention the specific week number in your response.\n"

    f"'Last week' refers to Week {last_week} of the season, and 'next week' refers to Week {next_week}.\n"

    "If asked to predict performance for 'next week', base your prediction on the player's skill level, "
    "recent performances, and overall season trends.\n"

    "Your 'next week' predictions should include key metrics such as passing yards, passing touchdowns, rushing yards, "
    "and total fantasy points. Ensure your estimates are reasonable based on past data.\n"
)

In [115]:
prompt = f"Use the following information to answer the query:\n\n{context}\n\nUser Query: {user_query}\n\nAnswer:"

# print(prompt)

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
)

print(completion.choices[0].message.content)


In Week 1 of the season, Baker Mayfield, playing as quarterback for the Tampa Bay Buccaneers, had an impressive performance. Here are his detailed statistics:

- Dropbacks: 34
- Pass Attempts: 30
- Completions: 24
- Passing Yards: 289
- Passing Touchdowns: 4
- Interceptions: 0
- Sacks: 1
- Rushing Attempts: 3
- Rushing Yards: 21
- Rushing Touchdowns: 0
- Fumbles: 0
- Fantasy Points: 29.7

This performance was notable for its high efficiency and productivity, as evidenced by his high completion rate and multiple touchdowns, contributing to a significant fantasy score of 29.7 points.


In [74]:
# completion = client.chat.completions.create(
#   model="gpt-4o-mini",
#   store=True,
#   messages=[
#     {"role": "user", "content": "How did Saquon Barkley do in the 2024 season?"}
#   ]
# )

# print(completion.choices[0].message);
